In [104]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer

---
## 讀取資料

In [3]:
df = pd.read_csv('movies_metadata.csv')
df.T # Transpose 轉置, 可把欄位全部顯示在左側

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2,3,4,5,6,7,8,9,...,45456,45457,45458,45459,45460,45461,45462,45463,45464,45465
adult,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",NaN,"{'id': 96871, 'name': 'Father of the Bride Col...",NaN,NaN,NaN,NaN,"{'id': 645, 'name': 'James Bond Collection', '...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
budget,30000000,65000000,0,16000000,0,60000000,58000000,0,35000000,58000000,...,0,0,0,0,0,0,0,0,0,0
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'id': 35, 'name': 'Comedy'}]","[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...","[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",...,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...","[{'id': 9648, 'name': 'Mystery'}, {'id': 27, '...","[{'id': 27, 'name': 'Horror'}]","[{'id': 878, 'name': 'Science Fiction'}]","[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...","[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...","[{'id': 18, 'name': 'Drama'}]","[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",[],[]
homepage,http://toystory.disney.com/toy-story,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.mgm.com/view/movie/757/Goldeneye/,...,NaN,NaN,NaN,NaN,NaN,http://www.imdb.com/title/tt6209470/,NaN,NaN,NaN,NaN
id,862,8844,15602,31357,11862,949,11860,45325,9091,710,...,84419,390959,289923,222848,30840,439050,111109,67758,227506,461257
imdb_id,tt0114709,tt0113497,tt0113228,tt0114885,tt0113041,tt0113277,tt0114319,tt0112302,tt0114576,tt0113189,...,tt0038621,tt0265736,tt0252966,tt0112613,tt0102797,tt6209470,tt2028550,tt0303758,tt0008536,tt6980792
original_language,en,en,en,en,en,en,en,en,en,en,...,en,en,en,en,en,fa,tl,en,en,en
original_title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,...,House of Horrors,Shadow of the Blair Witch,The Burkittsville 7,Caged Heat 3000,Robin Hood,رگ خواب,Siglo ng Pagluluwal,Betrayal,Satana likuyushchiy,Queerama
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...,"Cheated on, mistreated and stepped on, the wom...",Just when George Banks has recovered from his ...,"Obsessive master thief, Neil McCauley leads a ...",An ugly duckling having undergone a remarkable...,"A mischievous young boy, Tom Sawyer, witnesses...",International action superstar Jean Claude Van...,James Bond must unmask the mysterious head of ...,...,An unsuccessful sculptor saves a madman named ...,"In this true-crime documentary, we delve into ...",A film archivist revisits the story of Rustin ...,It's the year 3000 AD. The world's most danger...,"Yet another version of the classic epic, with ...",Rising and falling between a man and woman.,An artist struggles to finish his work while a...,"When one of her hits goes wrong, a professiona...","In a small town live two brothers, one a minis...",50 years after decriminalisation of homosexual...


---
## 資料清理

In [4]:
df.isna().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [9]:
df.loc[:10, ['title', 'overview']]

,title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...
5,Heat,"Obsessive master thief, Neil McCauley leads a ..."
6,Sabrina,An ugly duckling having undergone a remarkable...
7,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses..."
8,Sudden Death,International action superstar Jean Claude Van...
9,GoldenEye,James Bond must unmask the mysterious head of ...


In [10]:
df.loc[:, ['title', 'overview']] = df.loc[:, ['title', 'overview']].fillna('')

In [11]:
df.isna().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                     0
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        0
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [41]:
def get_title_overview(df):
    title_overviews = df.apply(lambda x: x['title'].lower() + ' ' + x['overview'].lower(), axis=1)
    title_overviews = title_overviews.values
    title_overviews = [re.sub(r'[^a-zA-Z ]+', ' ', title_overview) for title_overview in title_overviews]
    title_overviews = [title_overview.split(' ') for title_overview in title_overviews]
    return title_overviews

In [42]:
title_overviews = get_title_overview(df)
title_overviews[:5]

[['toy',
  'story',
  'led',
  'by',
  'woody',
  '',
  'andy',
  's',
  'toys',
  'live',
  'happily',
  'in',
  'his',
  'room',
  'until',
  'andy',
  's',
  'birthday',
  'brings',
  'buzz',
  'lightyear',
  'onto',
  'the',
  'scene',
  '',
  'afraid',
  'of',
  'losing',
  'his',
  'place',
  'in',
  'andy',
  's',
  'heart',
  '',
  'woody',
  'plots',
  'against',
  'buzz',
  '',
  'but',
  'when',
  'circumstances',
  'separate',
  'buzz',
  'and',
  'woody',
  'from',
  'their',
  'owner',
  '',
  'the',
  'duo',
  'eventually',
  'learns',
  'to',
  'put',
  'aside',
  'their',
  'differences',
  ''],
 ['jumanji',
  'when',
  'siblings',
  'judy',
  'and',
  'peter',
  'discover',
  'an',
  'enchanted',
  'board',
  'game',
  'that',
  'opens',
  'the',
  'door',
  'to',
  'a',
  'magical',
  'world',
  '',
  'they',
  'unwittingly',
  'invite',
  'alan',
  '',
  '',
  'an',
  'adult',
  'who',
  's',
  'been',
  'trapped',
  'inside',
  'the',
  'game',
  'for',
  '',
  '',

---
## 計算詞頻並排序

In [43]:
# 轉一維資料
hstack_arr = np.hstack(title_overviews) # hstask: stack horizontally
hstack_arr

array(['toy', 'story', 'led', ..., 'th', 'century', ''], dtype='<U27')

In [44]:
# 找出不一樣的字，並看 array 大小
np.unique(hstack_arr).shape

(78760,)

In [48]:
max_words = 10000

In [56]:
counter = Counter(hstack_arr)
counter = counter.most_common(max_words)
counter

[('', 341657),
 ('the', 153408),
 ('a', 102075),
 ('and', 77542),
 ('of', 74962),
 ('to', 74822),
 ('in', 50522),
 ('is', 36922),
 ('his', 36342),
 ('s', 29881),
 ('with', 24311),
 ('he', 22251),
 ('her', 21635),
 ('for', 19051),
 ('on', 18301),
 ('an', 17054),
 ('by', 16015),
 ('that', 15858),
 ('who', 15212),
 ('as', 14890),
 ('their', 13210),
 ('from', 12955),
 ('but', 11171),
 ('when', 10914),
 ('they', 10796),
 ('she', 10681),
 ('has', 9557),
 ('are', 9162),
 ('at', 9136),
 ('it', 9097),
 ('him', 8395),
 ('life', 8273),
 ('one', 7893),
 ('this', 7513),
 ('into', 6757),
 ('up', 6720),
 ('young', 6608),
 ('after', 6522),
 ('man', 6443),
 ('out', 6422),
 ('film', 6054),
 ('be', 5978),
 ('love', 5875),
 ('new', 5865),
 ('two', 5468),
 ('about', 5279),
 ('all', 5258),
 ('world', 5071),
 ('story', 4982),
 ('family', 4913),
 ('have', 4527),
 ('them', 4505),
 ('will', 4246),
 ('was', 4217),
 ('old', 3996),
 ('time', 3903),
 ('not', 3886),
 ('woman', 3741),
 ('only', 3609),
 ('find', 3594)

In [68]:
counter = {unique_word: idx for idx, (unique_word, count) in enumerate(counter)}
counter

{'': 0,
 'the': 1,
 'a': 2,
 'and': 3,
 'of': 4,
 'to': 5,
 'in': 6,
 'is': 7,
 'his': 8,
 's': 9,
 'with': 10,
 'he': 11,
 'her': 12,
 'for': 13,
 'on': 14,
 'an': 15,
 'by': 16,
 'that': 17,
 'who': 18,
 'as': 19,
 'their': 20,
 'from': 21,
 'but': 22,
 'when': 23,
 'they': 24,
 'she': 25,
 'has': 26,
 'are': 27,
 'at': 28,
 'it': 29,
 'him': 30,
 'life': 31,
 'one': 32,
 'this': 33,
 'into': 34,
 'up': 35,
 'young': 36,
 'after': 37,
 'man': 38,
 'out': 39,
 'film': 40,
 'be': 41,
 'love': 42,
 'new': 43,
 'two': 44,
 'about': 45,
 'all': 46,
 'world': 47,
 'story': 48,
 'family': 49,
 'have': 50,
 'them': 51,
 'will': 52,
 'was': 53,
 'old': 54,
 'time': 55,
 'not': 56,
 'woman': 57,
 'only': 58,
 'find': 59,
 'father': 60,
 'years': 61,
 'which': 62,
 'can': 63,
 't': 64,
 'where': 65,
 'while': 66,
 'get': 67,
 'girl': 68,
 'what': 69,
 'war': 70,
 'year': 71,
 'first': 72,
 'back': 73,
 'wife': 74,
 'finds': 75,
 'town': 76,
 'other': 77,
 'day': 78,
 'lives': 79,
 'home': 80,
 

---
## 製作 sklearn TF-IDF 所需的 Count Matrix

In [86]:
count_matrix = np.zeros([len(title_overviews), max_words], dtype=np.int32)
count_matrix.dtype # default=float64 較佔空間

dtype('int32')

In [87]:
count_matrix.shape

(45466, 10000)

In [88]:
count_matrix[0][0]

0

In [89]:
for idx, title_overview in enumerate(title_overviews):
    for word in title_overview:
        if word in counter:
            count_matrix[idx][counter[word]] += 1  
            # count_matrix[該電影index][該字的名次位置]，電影介紹每個字都跑一次，有出現在前萬名的話就在該字的名次上＋１

In [90]:
print(len(count_matrix[0]))

10000


In [91]:
count_matrix

array([[ 6,  2,  0, ...,  0,  0,  0],
       [11,  3,  1, ...,  0,  0,  0],
       [ 5,  4,  4, ...,  0,  0,  0],
       ...,
       [ 3,  0,  4, ...,  0,  0,  0],
       [11, 15,  5, ...,  0,  0,  0],
       [ 7,  5,  0, ...,  0,  0,  0]], dtype=int32)

---
## 製作 TF-IDF Matrix

In [99]:
transformer = TfidfTransformer()
tfidf_matrix = transformer.fit_transform(count_matrix)

In [100]:
tfidf_matrix.shape

(45466, 10000)

In [101]:
tfidf_matrix = tfidf_matrix.toarray()
tfidf_matrix

array([[0.11258727, 0.04278952, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.24846696, 0.07726212, 0.02619524, ..., 0.        , 0.        ,
        0.        ],
       [0.12060101, 0.11000447, 0.11188897, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.12010875, 0.        , 0.18572045, ..., 0.        , 0.        ,
        0.        ],
       [0.16330276, 0.25389929, 0.08608296, ..., 0.        , 0.        ,
        0.        ],
       [0.24290709, 0.19782517, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [102]:
(tfidf_matrix ** 2).sum(axis=1)  # 即 |A| * |B|＝餘弦相似度計算公式的分母，所以只要考慮 AB向量的內積即可 

array([1., 1., 1., ..., 1., 1., 1.])

---
## 推薦商品